In [10]:
import base64
import email
import os
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

# Load the trained model
with open('spam_model.pkl', 'rb') as f:
    spam_model = pickle.load(f)

# Define a predict function
def predict(text):
    # Create a pandas dataframe with the given text
    df = pd.DataFrame({'text': [text]})
    # Create a word count column using numpy's vectorize method
    df['word_count'] = np.vectorize(len)(df['text'].str.split())
    # Make predictions on the data using the trained model
    p = spam_model.predict_proba(df)[:,1]
    return p[0]

def spam(ids, addLabel="Test", removeLabel="INBOX"):
    batch_modify = {
        "ids" : [ids],
        "addLabelIds" : [addLabel],
        "removeLabelIds" : [removeLabel]
    }
    return batch_modify


def main():
    
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
   
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'token.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    try:
        results=service.users().messages().list(userId='me',maxResults=50,labelIds=['INBOX']).execute()
        messages=results.get('messages',[])
        for message in messages:
            msg=service.users().messages().get(userId='me',id=message['id']).execute()
            text=msg['snippet']
            if predict(text)<=0.5:
                new_label=spam(message['id'])
                try:
                    new_batch=service.users().messages().batchModify(userId='me',body=new_label).execute()
                except HttpError as error:
                    print(f"{error}")
    except TypeError as e:
        print("Error occured : {e}")



main()
print("Detection Complete") 

Detection Complete
